Hier könnte Ihre Werbung stehen.

In [1]:
import subprocess
import numpy as np
import sfs

#### testing the main execNEST function

In [2]:

if [False,True][0]:


    execNEST_ndarray = sfs.execNEST(
        spectrum_dict = {
#            "numEvts" : [10, 10, 10, 10],
            "numEvts" : 10,
            "type_interaction" : "ER",
#            "E_min[keV]" : [100, 100, 100, 100],
#            "E_max[keV]" : [100, 100, 100, 100],
            "E_min[keV]" : 100,
            "E_max[keV]" : 100,
            "field_drift[V/cm]" : "200",
        },
#        detector_dict = {},
        detector_dict = {
            "g1" : 1.9,
            "g1_gas" : 1.9,
        },
        detector_name = "rndm_new_detector",
        flag_verbose = True,
        flag_print_stdout_and_stderr = False,
    )


    #print(len(execNEST_ndarray))


#### compute g2 from detector configuration

In [20]:

if [False,True][1]:


    def convert_detector_header_into_detector_dict(
        abspathfile_detector_header,
    ):

        """
        This function is used to extract the detector parameters from a detector header file and to write them into a detector dictionary.
        """

        # initialization
        line_list = []
        detector_dict = {}
        parameter_list = [
            "g1", "sPEres", "sPEthr", "sPEeff", "noiseBaseline[0]", "noiseBaseline[1]", "noiseBaseline[2]",
            "noiseBaseline[3]", "P_dphe", "coinWind", "coinLevel", "numPMTs", "OldW13eV", "noiseLinear[0]",
            "noiseLinear[1]", "g1_gas", "s2Fano", "s2_thr", "E_gas", "eLife_us", "T_Kelvin", "p_bar", "dtCntr",
            "dt_min", "dt_max", "radius", "radmax", "TopDrift", "anode", "gate", "cathode", "PosResExp", "PosResBase"]

        # reading the lines of the input file
        with open(abspathfile_detector_header, 'r') as inputfile:
            for line in inputfile:
                line_list.append(line)

        # extracting the detector parameters into 
        for line in line_list:
            if "=" in line and any([param in line for param in parameter_list]) and ";" in line:
                line_mod = line
                for k in range(0,50):
                    line_mod = line_mod.replace("  "," ")
                line_mod_list = list(line_mod.split(" "))
                extracted_parameter = line_mod_list[1]
                extracted_value = line_mod_list[-1][:-2] if extracted_parameter=="OldW13eV" else float(line_mod_list[-1][:-2])
                detector_dict.update({extracted_parameter:extracted_value})

        return detector_dict


    def compute_g2_from_detector_configuration(
        detector_dict, # dict, detector configuration
    ):
        """
        This function is used to compute the g2 value of a specific detector configuration.
        The comments on the official repo .
        """
        
        EPS_GAS = 1.00126
        EPS_LIQ = 1.85 # instead of 1.96 as in old NEST versions
        E_liq = detector_dict["E_gas"]/(EPS_LIQ/EPS_GAS)
        T_Kelvin = detector_dict["T_Kelvin"]
        
        em1 = 8.807528626640e4 -2.026247730928e3*T_Kelvin +1.747197309338e1*T_Kelvin**2 -6.692362929271e-2*T_Kelvin**3 +9.607626262594e-5*T_Kelvin**4
        em2 = 5.074800229635e5 -1.460168019275e4*T_Kelvin +1.680089978382e2*T_Kelvin**2 -9.663183204468e-1*T_Kelvin**3 +2.778229721617e-3*T_Kelvin**4 -3.194249083426e-6*T_Kelvin**5
        em3 = -4.659269964120e6 +1.366555237249e5*T_Kelvin -1.602830617076e3*T_Kelvin**2 +9.397480411915e-0*T_Kelvin**3 -2.754232523872e-2*T_Kelvin**4 +3.228101180588e-5*T_Kelvin**5
        ExtEff = 1 - em1 *np.exp(-em2 *(E_liq**em3))
        #ExtEff = 1 if ExtEff >= 1
        #ExtEff = 0 if ExtEff < 0
        SE_size = detector_dict["g1_gas"]*(detector_dict["TopDrift"]-detector_dict["gate"])*0.1*(0.137*detector_dict["E_gas"]*1000-177*detector_dict["p_bar"]-45.7)
        g2 = ExtEff*SE_size
        
        print(f"ExtEff={ExtEff}")
        print(f"SE_size={SE_size}")
        
        return g2


    #detector_dict_test = sfs.darwin_baseline_detector_dict.copy()
    #detector_dict_test.update({
    #    "g1" : 1.9,
    #    "g1_gas" : 1.9,
    #})
    #print(detector_dict_test)
    
    
    
    extracted_detector_dict = convert_detector_header_into_detector_dict(sfs.abspath_nest_installation_install +"include/Detectors/rndm_new_detector.hh")
    g2_test = compute_g2_from_detector_configuration(
        detector_dict = extracted_detector_dict
    )
    print(g2_test/(1 +extracted_detector_dict["P_dphe"]))
    
    

ExtEff=0.5430850127427067
SE_size=581.9310500000001
269.42713700309184


In [15]:
print(0.543085 *435.384)

236.45051964000004
